# Spatial Density Estimation for Tackle Success Using Deep Convolutional Gauccian-Mixture Ensembling 
#### Cumulative Yards Saved (CYS) & Sphere of Influence (SOI): Metrics for Tackler Efficiencies & Role Classification

# Introduction

Aside from a few beat-up punt returners, running backs, and most recently poorly protected quarterbacks, you’d be hard-pressed to find anyone who has been within throwing distance of a football stadium who wouldn’t argue that “the tackle” is what makes football, football. While the idea of eleven grown men all aiming to bring down one ball carrier is hard to frame as a calculated science, it certainly is one. Research for injury prevention, for instance, is already a focus for the NFL and will continue to progress through studying tackle-related contact. What is less explored, though, is stratifying responsibility for tackles, and recognizing which players directly or indirectly contribute to saved yardage when a scramble, run, or run after catch begins. 

Using an ensemble of ten convolutional networks trained on independent play frames, I created a model (TackleNet) that predicts the spatial distribution of where the “end-of-play” will occur with confidence estimates. Additionally, with such predictions and confidences, I made two metrics, Cumulative Expected Yards Saved (CEYS) and Degree of Direct Influence. Below is a visual demonstration of the result - read on if you're interested in how its done.

<table style='margin: 0 auto; border: 0;'>
  <tr>
    <td style="text-align: center;"> <img src="https://imgur.com/aYLjR2c.gif" alt="Drawing" style="width: 1000px;"/> </td>
    <td style="text-align: center;"> <img src="https://imgur.com/FHKCDRL.gif" alt="Drawing" style="width: 1000px;"/> </td>
</table>

# Main Points

* **A shortcoming of the current state of sports analytics is failing to recognize the limitations of our data and the degree of prediction confidence we may have.** Limited data means limited and **varying confidence depending on the in-play situation**, and we should account and report for these varying intervals. From play-call-decision analytics (see Ryan Brill's talk here) to  metrics like those in the BDB, we should start reporting metric confidence. **We estimate variance in prediction by using ensemble model methods for our spatial densities.**

* Tackling encompasses more than big hits: **valuable tacking skills include any type of coersion that reduces the ball carriers yardage** by the end of the play. This can be direct, like the highlight hits you'll see on replay, or indirect like having positioning that manipulates the ball carrier into a worse route or out of bounds. We should have ways of measuring how direct a defenders influence is spatially, and how much he reduces yardage.

* **Spatial density estimation and the use of the subtraction method allows us to (1) where the tackler is coercing the ball carrier to go in the context of the rest of his team, and (2) how direct, or broad, his influence on the ball carrier is.**

# Modeling Strategy

### i. TackleNet

Continuous density estimation can be difficult. Often we simplify our state space by imagining continuous spaces, like yardage on a football field, as discrete areas. While this can be useful, it loses us a lot of information and causes our model output to be really high dimensional (think 54*120 gridded yard areas). Enter deep density mixture models, which are extremely ideal for estimating a distribution of results from inputs. The general idea revolves around the concept that **we can model our output, in this case where the tackle or end of play occured, by modeling the parameters of an assumed mixture distribution with deep learning. In our case, the normal mixture distribution works perfectly. Why?**

<div style="text-align: center; margin-top: 40px; margin-bottom: 40px;">
  <blockquote>
    "The mechanics of defensive effort to tackle a ball carrier match the intuition of the normal mixture model. Each tackle attempt is either successful or not - if it is, then the successful tackle will occur around some mean with variance dependent on the yards after contact - if it's not, then the successful tackle will occur downfield at some other mean with variance dependent on different yards after contact."
  </blockquote>
</div>

Well discuss our input features later, but its evident with a spatially dependent problem like tackle location estimation that we need features which account for such dependencies spatially. Mapping velocities and movement features via images turns out to be a great way of doing this, and allows us to embed all our information about a play instance into a vector feature set that we can use for estimating distribution parameters using convolutional autoencoding - but well get into that later. **We can use this auto-encoded embedding of 'player movement maps' to feed into three fully connected networks for each parameter of the normal distribution**: one for the means of the mixture distribution, one for the covariance matrix (which we assume to be diagonal for some degree of simplicity), and one for the mixture probabilities. All of this comes together to make TackleNet, which is diagrammed below.

<div>
<center><img src="https://imgur.com/LXQEqpb.png" width="800px;"/><center>
</div>

### ii. Ensemble TackleNet

If we take a step back and think about the logistics of training this mixture model, we still need to fix a few issues.

1. We need to find a way not to overfit the data by training on a bunch of dependent play situations. This means, **ideally, we should only train a model on one frame per play as to not rely on situational dependence driving our supposedly independent observations from these gauccian densities.** 

2. As foreshaddowed previously, **we certainly need to find a way to quantify our density prediction confidence for any given observation, as 9 weeks of play data (~12000 plays) is admittedly only a snapshot of what we would need to be precisely confident for density prediction in all tackle situations.**

The solution of ensemble modeling (i.e. training ten different TackleNet models on bagged frame samples limited on from each play), is a perfect solution. **Ensembling allows us to only use independent samples in model training, and also still use a much more vast portion of the dataset available to use and also allows us to pull spatial prediction intervals confidence intervals from the range of prediction densities** outputted. Additionally, and conveniently, **an ensemble of gauccian mixture models is also a gauccian mixture model, just with more mixtures**.



<div>
<center><img src="https://imgur.com/J1vnbfe.png" width="800px;"/><center>
</div>


<div>
<center><img src="https://imgur.com/jiRFD7D.png" width="800px;"/><center>
</div>

### iii. Feature Selection & Engineering


To account for the spatial correlation with the convolutional autoencoder decribed above, we first need to determine featuere, or 'movement,' maps that well capture the variability of end-of-play tackle outcomes. To do so, I created images describing the velocities, accelerations, and positioning of the offensive and defensive groups as a whole in various directions and areas. The figure below decribes such feature maps, which are weighted by both distance from the ballcarrier and distance from the area described, and make up the 8 channel input to TackleNet.


<table style='margin: 0 auto; border-collapse: collapse; width: 100%;'>
  <tr>
    <td style="text-align: center;"> <img src="https://imgur.com/Tf7S8MO.gif" alt="Drawing" style="width: 600px;"/> </td>
    <td style="text-align: center;"> <img src="https://imgur.com/9RLgF1W.png" alt="Drawing" style="width: 300px;"/> </td>
    <td style="text-align: center;"> <img src="https://imgur.com/KQKTBZC.png" alt="Drawing" style="width: 300px;"/> </td>
  </tr>
</table>


### iv. Training Notes & Results

After initially training models with gauccian log loss, I recognized a problem. As with class imbalance problem with a relatively low sample size, often the best maximum liklihood solution is to guess the majority class most, or all, of the time - but this does little good for utilities sake. A similar issue was occuring here, on more continuous scale: **due to the low sample size and "class imbalance" with play yardage result (i.e. there are many short yardage gains and few breakaway plays), the maximum liklihood distributions favored placing modes near the ball carrier, and not accounting for downfield chances.** 

I considered a couple solutions here, including adding bayesian priors account for chances in larger gain plays, but **settled on instead using a custom "weighted" log loss to weight breakout plays more heavily (and also other play yardages that occured less frequently). While this certainly intoduces some bias to our distribution, in our small sample setting (at least for density estimation) its much more useful to overpredict the chance of anamoly type plays than to not assign them any probability at all.**

<div>
<center><img src="https://imgur.com/Z1u4Z3W.png" width=800px;"/><center>
</div>






# Applications: Metric Creation

The potential applications of TackleNet and similar methods to estimate spatial distributions of successful tackles in real time are massive, and there is a ton of unexplored directions to explore beyond the scope of this paper. Many of the ideas that enspired the creation of TackleNet include live broadcast ideas and non-metric related applications, but they can also help us stratify responsibility and tackle roles on plays.

<div>
<center><img src="https://imgur.com/n2Hhts6.gif" width="1500px;"/><center>
</div>





# Code

Review code [here on GitHub](https://github.com/nickb1125/Big-Data-Bowl-Tackles).